In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,Dropout,LSTM
from keras.layers import  CuDNNLSTM 
from keras.models import Model
from keras.utils import to_categorical

#model selection
from sklearn.model_selection import train_test_split

In [ ]:
import os 
print(os.listdir(r'../input/'))

In [ ]:
train = pd.read_csv(r"../input/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

test = pd.read_csv(r"../input/testData.tsv", header=0, delimiter="\t", quoting=3)

In [ ]:
print("the train shape is : ",train.shape)
print("the test shape is : ",test.shape)

In [ ]:
df=train.copy()

In [ ]:
df.head()

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df['review'][0]

In [ ]:
test.head()

In [ ]:
test['review'][0]

In [ ]:
test.head()

In [ ]:
# pre-processing starts

In [ ]:
def clean_reviews(review):
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    stop_words= set(stopwords.words("english"))     
    word_tokens= [w for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

In [ ]:
df['review']=df['review'].apply(clean_reviews)
test['review']=test['review'].apply(clean_reviews)

In [ ]:
print("the train shape is : ",df.shape)
print("the test shape is : ",test.shape)

In [ ]:
df['review'][0]

In [ ]:
test['review'][0]

In [ ]:
# encoding starts

In [ ]:
'''

need to find no of unique words.

'''
master=[]
for e in df['review']:
  master.append(e)

words=[]
for review in master:
  tok=review.split()
  for r in tok:
    words.append(r)
    
master=[]
for e in test['review']:
  master.append(e)
  
for review in master:
  tok=review.split()
  for r in tok:
    words.append(r)
    
print(len(set(words)))
  
vocab_size=len(set(words))


In [ ]:
tokenizer=Tokenizer(num_words=vocab_size)
a=tokenizer.fit_on_texts(list(df['review']))
encoded_train=tokenizer.texts_to_sequences(list(df['review']))

tokenizer=Tokenizer(num_words=vocab_size)
a=tokenizer.fit_on_texts(list(test['review']))
encoded_test=tokenizer.texts_to_sequences(list(test['review']))

print()


In [ ]:
'''

need to find the maximum no of words in any document. 

we will pad the shorter documents with zeros. 

all the document should be of same length. this will be required by the Embedding layer in keras.

'''

maxi=-1
for str in df['review']:
  tokens=nltk.word_tokenize(str)
  if(maxi<len((tokens))):
    maxi=len(tokens)
    
for str in test['review']:
  tokenss=nltk.word_tokenize(str)
  if(maxi<len((tokens))):
    maxi=len(tokens)
    
print(maxi)

In [ ]:
# therefore we need to encode each document to consisting of 1416 words
max_doc_len=maxi
pad_encoded_train=pad_sequences(encoded_train,max_doc_len)
pad_encoded_test=pad_sequences(encoded_test,max_doc_len)

Y=df['sentiment'].values
Y=to_categorical(Y,2)

print(pad_encoded_train.shape)

In [ ]:
# ready for creating embeddings.

In [142]:
vocab_size=vocab_size
embed_size=300
max_doc_len=maxi

In [143]:
from keras.layers import Bidirectional, GlobalMaxPool1D
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_size,input_length=max_doc_len))
model.add(CuDNNLSTM(32, return_sequences =False))
model.add(Dropout(0.40))
model.add(Dense(16, activation="relu"))
model.add(Dropout(0.40))
model.add(Dense(2, activation="softmax"))

In [144]:
# compiling the model. parameters can be tuned as always.
model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc'])

In [145]:
model.summary() # summary of the model.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 1416, 300)         30373500  
_________________________________________________________________
cu_dnnlstm_16 (CuDNNLSTM)    (None, 32)                42752     
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_12 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 34        
Total params: 30,416,814
Trainable params: 30,416,814
Non-trainable params: 0
________________________________________________________________

In [146]:
batch_size=128
epochs=5
x_train,x_test,y_train,y_test=train_test_split(pad_encoded_train,Y,test_size=0.20,random_state=42)
y_test.shape

(5000, 2)

In [ ]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))

In [ ]:
pred=embed_model.predict(pad_encoded_test)
pred=np.argmax(pred,axis=1)
d={'id':test['id'],'sentiment':pred}
answer=pd.DataFrame(d)
answer.to_csv('answer.csv',index=False)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
pred

In [ ]:
arr=[0,0]
for i in pred:
   arr[i]=arr[i]+1
print(arr)

In [ ]:
arr=[0,0]
for ty in y_test:
  if ty[0]==1:
    arr[0]=arr[0]+1
  else:
    arr[1]=arr[1]+1
print(arr)